In [21]:
from openai import OpenAI 
import yaml
import os
from dotenv import load_dotenv

load_dotenv()

MODEL="gpt-4o"
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY", "<your OpenAI API key if not set as an env var>")
DB_INDEX_FILE_LOCAL_PATH = os.environ.get("DB_INDEX_FILE_LOCAL_PATH", "<path to the index.json file>")
DB_ROOT_LOCAL_PATH = os.environ.get("DB_ROOT_LOCAL_PATH", "<path to the root directory of the database>")

openAiClient = OpenAI(api_key=OPENAI_API_KEY)

with open("../prompts/memoryClassifier.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt_for_classifier = group_classifier["prompt"]

with open("../prompts/memoryAssistant.yml", "r") as file:
    group_classifier = yaml.safe_load(file)

system_prompt_for_assistant = group_classifier["prompt"]

with open(DB_INDEX_FILE_LOCAL_PATH, "r") as file:
    metadata = file.read()

In [22]:
def create_messages_for_classifier(query, metadata):
    return [
        {
            "role": "system",
            "content": system_prompt_for_classifier
        },
        {
            "role": "user",
            "content": f"Query: ${query}"
        },
        {
            "role": "user",
            "content": f"Metadata: ${metadata}"
        }        
    ]

def create_messages_for_assistant(query, memories):
    return [
        {
            "role": "system",
            "content": system_prompt_for_assistant
        },
        {
            "role": "user",
            "content": f"Query: ${query}"
        },
        {
            "role": "user",
            "content": f"Memories: ${memories}"
        }
    ]

In [23]:
query = "any memory about vacation"
completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages_for_classifier(query, metadata)),

completion_content = completion[0].choices[0].message.content
print(completion_content)

{
  "thinking_process": "The user's query is about memories related to vacations. Based on the provided metadata, the relevant category for this query is 'Areas' as it includes long-term responsibilities and interests, which can encompass travel and vacation plans. Specifically, the 'AREAS/TRAVELLING' files contain detailed information about various travel destinations and plans, which are directly related to the user's query about vacation memories.",
  "keys": [
    "AREAS/TRAVELLING/galicia",
    "AREAS/TRAVELLING/roadmap_23-24",
    "AREAS/TRAVELLING/travelling"
  ]
}


In [24]:
# parse the response in JSON and print the value of *data*
import json


keys = json.loads(completion_content)["keys"]

memories = []
for key in keys:
    memory_asset_path = os.path.join(DB_ROOT_LOCAL_PATH, key)
    with open(memory_asset_path, "r") as file:
        memory_asset_content = file.read()
        memories.append({
            "path": key,
            "content": memory_asset_content
        })

memories_string = json.dumps(memories)

completion = openAiClient.chat.completions.create(
    model=MODEL,
    temperature=0,
    stream=False,
    response_format={"type": "json_object"},
    messages=create_messages_for_assistant(query, memories_string)),

completion_content = completion[0].choices[0].message.content
print(completion_content)    


{
  "thinking_process": "The user is looking for memories related to vacations. I will search through the provided memories to find any relevant information about past or planned vacations. The memories include various travel destinations and a roadmap for travel plans in 2023-2024.",
  "response": "Here are some vacation-related memories based on your notes:\n\n1. **Galicia**: You have a memory about visiting Galicia, specifically A Coruña, Anos, and A Eirita Candas Rosamonde.\n\n2. **Travel Roadmap 2023-2024**: You have planned several trips for the upcoming years:\n   - **December 2023**: New York\n   - **January 2024**: Salem\n   - **February 2024**: Rome (twice)\n   - **March 2024**: Åre/Trisil\n   - **April 2024**: Cabin (possibly with Marta) and considering destinations like Sweden, Brazil, or France\n   - **May 2024**: Taylor in Sweden\n   - **June/July 2024**: Sweden\n   - **July 2024**: Milan, Taylor, and Dolomites\n   - **August 2024**: Argentina (Bayo)\n   - **September 202